## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import json
import requests
import os
from pprint import pprint
from dotenv import load_dotenv
load_dotenv()
g_key = os.getenv('g_key')
type(g_key)

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mar Del Plata,AR,-38.0023,-57.5575,51.10,94,81,4.00,broken clouds
1,1,Zhigansk,RU,66.7697,123.3711,72.18,51,25,3.40,scattered clouds
2,2,Tuktoyaktuk,CA,69.4541,-133.0374,39.29,84,46,7.54,scattered clouds
3,3,Port Elizabeth,ZA,-33.9180,25.5701,50.09,67,81,4.92,broken clouds
4,4,Mahdasht,IR,35.7299,50.8111,82.62,45,4,19.84,clear sky


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Zhigansk,RU,66.7697,123.3711,72.18,51,25,3.40,scattered clouds
6,6,Saint-Philippe,RE,-21.3585,55.7679,74.21,74,17,8.19,light rain
9,9,Vaini,TO,-21.2000,-175.2000,72.46,68,48,20.54,scattered clouds
17,17,Hilo,US,19.7297,-155.0900,79.29,92,48,1.99,scattered clouds
23,23,Bambous Virieux,MU,-20.3428,57.7575,73.22,66,40,11.23,scattered clouds
26,26,Samana,DO,19.2056,-69.3369,78.28,85,14,10.51,few clouds
28,28,Albany,US,42.6001,-73.9662,72.73,88,100,10.96,overcast clouds
29,29,Rikitea,PF,-23.1203,-134.9692,75.40,80,13,18.90,few clouds
33,33,San Matias,BO,-16.3667,-58.4000,72.72,42,0,4.25,clear sky
34,34,Maroantsetra,MG,-15.4333,49.7333,70.03,84,75,5.44,light rain


In [11]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                207
City                   207
Country                207
Lat                    207
Lng                    207
Max Temp               207
Humidity               207
Cloudiness             207
Wind Speed             207
Current Description    207
dtype: int64

In [25]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Zhigansk,RU,72.18,scattered clouds,66.7697,123.3711,
6,Saint-Philippe,RE,74.21,light rain,-21.3585,55.7679,
9,Vaini,TO,72.46,scattered clouds,-21.2000,-175.2000,
17,Hilo,US,79.29,scattered clouds,19.7297,-155.0900,
23,Bambous Virieux,MU,73.22,scattered clouds,-20.3428,57.7575,
26,Samana,DO,78.28,few clouds,19.2056,-69.3369,
28,Albany,US,72.73,overcast clouds,42.6001,-73.9662,
29,Rikitea,PF,75.40,few clouds,-23.1203,-134.9692,
33,San Matias,BO,72.72,clear sky,-16.3667,-58.4000,
34,Maroantsetra,MG,70.03,light rain,-15.4333,49.7333,


In [23]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()  
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
        
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")  

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
-----------------------------
Data Retrieval Complete      
-----------------------------


In [28]:
# 7. Drop the rows where there is no Hotel Name.
#clean_hotel_df = hotel_df.dropna(axis="index", how="any")
#clean_hotel_df.head(10)

clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Zhigansk,RU,72.18,scattered clouds,66.7697,123.3711,Gostinichnyy Kompleks Aylgy
6,Saint-Philippe,RE,74.21,light rain,-21.3585,55.7679,Le four à pain
9,Vaini,TO,72.46,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
17,Hilo,US,79.29,scattered clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
23,Bambous Virieux,MU,73.22,scattered clouds,-20.3428,57.7575,Casa Tia Villa
26,Samana,DO,78.28,few clouds,19.2056,-69.3369,The Bannister - Hotel & Yacht Club
29,Rikitea,PF,75.40,few clouds,-23.1203,-134.9692,People ThankYou
33,San Matias,BO,72.72,clear sky,-16.3667,-58.4000,"HOTEL ""SAN JOSE"""
34,Maroantsetra,MG,70.03,light rain,-15.4333,49.7333,Annexe Vatsy Hôtel
41,Tiarei,PF,73.42,light rain,-17.5333,-149.3333,Le Rocher de Tahiti


In [29]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [37]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [38]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))